In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import time
import copy
import datetime
import numba
import timeit
import xgboost as xgb
from sqlalchemy import create_engine
from sklearn.metrics import roc_auc_score

In [2]:
engine = create_engine('postgresql://postgres:@192.168.99.100:5432/nhlstats')
conn = engine.connect()
np.random.seed(42)

In [3]:
df = pd.read_sql('team_stats_public',engine,parse_dates=['gameDate']).set_index('gameId').sort_index()

In [94]:
def slice_team(df,teamSlct,varCode,hist_len=10):
    newdf = {}
    cols = [teamSlct+'_'+varCode+'_L'+x for x in np.arange(1,hist_len+1)[::-1].astype('str')]
    for team in df[teamSlct].unique():
        hist_vals = df[df[teamSlct] == team][varCode].values
        hist_idx = df[df[teamSlct] == team].index
        for date in hist_idx:
            pos = np.searchsorted(hist_idx,date)
            slicer = hist_vals[:pos][-hist_len:]
            if len(slicer)<hist_len:
                slicer = np.insert(slicer,0,-999*np.ones(hist_len-len(slicer)))
            newdf[date] = pd.Series(slicer,index=cols)
    return(df.join(pd.DataFrame(newdf).T))

In [95]:
dd = slice_team(df,'teamAbbrevHome','goalsForHome')

In [96]:
dd

,faceoffWinPctgHome,faceoffsLostHome,faceoffsWonHome,gameDate,goalsAgainstHome,goalsForHome,penaltyKillPctgHome,ppGoalsAgainstHome,ppGoalsForHome,ppOpportunitiesHome,...,teamAbbrevHome_goalsForHome_L10,teamAbbrevHome_goalsForHome_L9,teamAbbrevHome_goalsForHome_L8,teamAbbrevHome_goalsForHome_L7,teamAbbrevHome_goalsForHome_L6,teamAbbrevHome_goalsForHome_L5,teamAbbrevHome_goalsForHome_L4,teamAbbrevHome_goalsForHome_L3,teamAbbrevHome_goalsForHome_L2,teamAbbrevHome_goalsForHome_L1
gameId,,,,,,,,,,,,,,,,,,,,,
2005020001,0.5645,27,35,2005-10-05 23:00:00,2,1,0.8000,1,0,7,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2005020002,0.4852,35,33,2005-10-05 23:00:00,4,6,0.8571,1,2,11,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2005020003,0.4843,33,31,2005-10-05 23:00:00,5,3,0.5000,2,1,10,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2005020004,0.4754,32,29,2005-10-05 23:00:00,2,3,1.0000,0,2,8,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2005020005,0.4761,33,30,2005-10-05 23:00:00,0,2,1.0000,0,1,8,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2005020006,0.4754,32,29,2005-10-06 00:00:00,2,2,1.0000,0,1,11,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2005020007,0.5714,24,32,2005-10-05 23:30:00,1,5,0.9090,1,2,8,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2005020008,0.4838,32,30,2005-10-05 23:30:00,2,5,1.0000,0,1,6,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2005020009,0.4821,29,27,2005-10-05 23:30:00,1,5,1.0000,0,1,7,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [70]:
@numba.jit
def slicer_jit(hist_vals,hist_idx,teamSlct,varCode,hist_len=10):
    returndf = {}
    # create columns
    cols = np.arange(1,hist_len+1)[::-1].astype('str')
    for i, item in enumerate(cols):
        cols[i] = teamSlct+'_'+varCode+'_L'+item
    # slicer itself
    for date in hist_idx:
        pos = np.searchsorted(hist_idx,date)
        slicer = hist_vals[:pos][-hist_len:]
        if len(slicer)<hist_len:
            slicer = list(-999*np.ones(hist_len-len(slicer))).append(slicer)
        returndf[date] = pd.Series(slicer,index=cols)
    return(returndf)

In [71]:
def slice_team_jit(df,teamSlct,varCode,hist_len=10):
    newdf = pd.DataFrame()
    for team in df[teamSlct].unique():
        hist_vals = df[df[teamSlct] == team][varCode].values
        hist_idx = df[df[teamSlct] == team].index
        rdf = slicer_jit(hist_vals,hist_idx,teamSlct,varCode)
        
    return(rdf)

In [77]:
dd = slice_team_jit(df,'teamAbbrevHome','goalsForHome')

In [79]:
kk = pd.DataFrame(dd)

In [83]:
kk

,2014020016,2014020030,2014020049,2014020072,2014020115,2014020183,2014020212,2014020229,2014020276,2014020320,...,2015020881,2015020963,2015020978,2015021059,2015021076,2015021097,2015021110,2015021127,2015021138,2015021177
teamAbbrevHome_goalsF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,1,3,2,4,1,4,6,6
teamAbbrevHome_goalsF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,3,2,4,1,4,6,6,4
teamAbbrevHome_goalsF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,3,2,4,1,4,6,6,4,1
teamAbbrevHome_goalsF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,2,4,1,4,6,6,4,1,5
teamAbbrevHome_goalsF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,4,1,4,6,6,4,1,5,3
teamAbbrevHome_goalsF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4,1,4,6,6,4,1,5,3,0
teamAbbrevHome_goalsF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,4,6,6,4,1,5,3,0,4
teamAbbrevHome_goalsF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4,6,6,4,1,5,3,0,4,3
teamAbbrevHome_goalsF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6,6,4,1,5,3,0,4,3,2
teamAbbrevHome_goalsF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6,4,1,5,3,0,4,3,2,2


In [64]:
cols=['L1','L2','L3','L4','L5']
vals=np.arange(5)
zip(cols,vals)

[('L1', 0), ('L2', 1), ('L3', 2), ('L4', 3), ('L5', 4)]

In [26]:
hist_vals = df[df['teamAbbrevHome'] == 'EDM']['goalsForHome'].values
hist_idx = df[df['teamAbbrevHome'] == 'EDM'].index
hist_idx

Int64Index([2005020014, 2005020036, 2005020070, 2005020092, 2005020111,
            2005020183, 2005020289, 2005020305, 2005020314, 2005020370,
            ...
            2015020936, 2015021000, 2015021026, 2015021036, 2015021049,
            2015021065, 2015021083, 2015021137, 2015021176, 2015021201],
           dtype='int64', name=u'gameId', length=445)

In [30]:
print(np.searchsorted(hist_idx,2005020470))

13


In [34]:
hist_vals[:13][-10:]

array([4, 3, 2, 3, 1, 5, 6, 3, 1, 2, 5, 4, 5])